# 3-MutationsToStructure
In this tutorial we create and join various datasets to analyse missense mutations.

1. Calculate protein-ligand interactions for [Imatinib](https://en.wikipedia.org/wiki/Imatinib) (Gleevec)
2. Join the results with the SIFTS dataset to obtain UniProt mappings
3. Use the UniProt mappings to retrieve pathogenic mutations from [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/)
4. Map the mutations from genome locations to 3D PDB positions
5. Visualize the locations of the mutations on the 3D structure

In [17]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import concat_ws
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset, myVariantDataset
from mmtfPyspark.filters import ContainsGroup
from mmtfPyspark.interactions import InteractionFilter, InteractionFingerprinter
from mmtfPyspark.io import mmtfReader
from ipywidgets import interact, IntSlider
import py3Dmol

#### Configure Spark Session

In [18]:
spark = SparkSession.builder.appName("Solution-1").getOrCreate()

#### Read a  sample of the PDB

In [19]:
path = "../resources/mmtf_full_sample"
pdb = mmtfReader.read_sequence_file(path)

## 1. Calculate protein-ligand interactions for Imatinib (Gleevec)  `STI`

In [20]:
pdb = pdb.filter(ContainsGroup(("STI")))

#### [InteractionFilter](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/interactions/interactionFilter.py) sets criteria for finding interactions

In [21]:
interactionFilter = InteractionFilter()
interactionFilter.set_distance_cutoff(4.5)
interactionFilter.set_min_interactions(1)
interactionFilter.set_query_groups(True, ["STI"])

#### [InteractionFingerprinter](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/interactions/interactionFingerprinter.py) finds ligand-polymer interactions that meet the filter criteria

In [22]:
interactions = InteractionFingerprinter.get_ligand_polymer_interactions(pdb, interactionFilter).cache()

In [23]:
interactions.toPandas().head(5)

Py4JJavaError: An error occurred while calling o241.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 1.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1.0 (TID 5, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:460)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:453)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:284)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$

## 2. Join the results with the SIFTS dataset to obtain UniProt mappings

### Get up-to-date PDB chain to UniProt mapping from the SIFTS projects

In [ ]:
uniprotQuery = "SELECT * FROM sifts.pdb_chain_uniprot"
uniprot = pdbjMineDataset.get_dataset(uniprotQuery)

In [ ]:
uniprot.toPandas().head(5)

### Map interacting chains to UniProt data

In [ ]:
intersection = uniprot.join(interactions, uniprot.structureChainId == interactions.structureChainId).cache()

In [ ]:
intersection.toPandas()

### Extract UniProt Ids as a list

In [ ]:
uniprot_ids = intersection.select("sp_primary").distinct().rdd.flatMap(lambda x: x).collect()
uniprot_ids

## 3. Use the UniProt mappings to retrieve pathogenic mutations from ClinVar
Here we use [myVariantDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/myVariantDataset.py) to retrieve variant information using [MyVariant.info](http://myvariant.info) web services ([Query syntax](http://myvariant.info/docs))

In [ ]:
query = "clinvar.rcv.clinical_significance:pathogenic"
variants = myVariantDataset.get_variations(uniprot_ids, query).cache()
variants.toPandas()

### Extract Variant Ids as a list

In [ ]:
variant_ids = variants.select("variationId").distinct().rdd.flatMap(lambda x: x).collect()
variant_ids

## 4. Map the mutations from genome locations to 3D PDB positions
Here we use [g2sDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/g2sDataset.py) to retrieve genome to PDB mapping information using the [G2S]( https://g2s.genomenexus.org) (Genome to Structure) web services.

In [ ]:
positions = g2sDataset.get_position_dataset(variant_ids).cache()
positions.toPandas()

#### Concatenate structureId and chainId columns to structureChainId (required for join operation below)
See documentation for [concat_ws](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.functions.concat_ws)

In [ ]:
positions = positions.withColumn("structureChainId", concat_ws(".", positions.structureId,positions.chainId))

In [ ]:
positions.toPandas().head(5)

### Join mutation data with the structures in the sample PDB set that interact with Imatinib

In [ ]:
positions = positions.join(interactions, positions.structureChainId == interactions.structureChainId)
positions.toPandas()

## 5. Visualize the locations of the mutations on the 3D structure

#### Create lists of ids

In [ ]:
pdb_ids = positions.select("structureId").rdd.flatMap(lambda x: x).collect()
chain_ids = positions.select("chainId").rdd.flatMap(lambda x: x).collect()
group_numbers = positions.select("pdbPosition").rdd.flatMap(lambda x: x).collect()
var_ids = positions.select("variationId").rdd.flatMap(lambda x: x).collect()

#### Custom method to visualize the site of a mutation

In [ ]:
def view_mutation_site(pdb_ids, chain_ids, groups, var_ids, ligand_id, distance=4.5):
    
    def view3d(i=0):
        
        print(f"PDB: {pdb_ids[i]}, chain: {chain_ids[i]}, group: {groups[i]}, variation: {var_ids[i]}")

        mutation = {'resi': groups[i], 'chain': chain_ids[i]}
        neighbors = {'resi': groups[i], 'chain': chain_ids[i], 'byres': 'true', 'expand': distance}

        viewer = py3Dmol.view(query='pdb:' + pdb_ids[i]) 
        viewer.setStyle(neighbors, {'stick': {'colorscheme': 'orangeCarbon', 'radius': 0.1}});
        viewer.setStyle(mutation, {'stick': {'colorscheme': 'redCarbon'}})
        viewer.addResLabels(mutation)
        viewer.setStyle({'resn': ligand_id}, {'sphere': {'colorscheme': 'greenCarbon'}})
        viewer.zoomTo(neighbors)

        return viewer.show()

    s_widget = IntSlider(min=0, max=len(pdb_ids)-1, description='Structure', continuous_update=False)
    return interact(view3d, i=s_widget)

#### The mutation site is rendered as a red stick. Residues within 8 A of the mutation site are rendered as thin orange stickes. Imatinib (Gleevec) is rendered in as spheres.

In [ ]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

In [ ]:
view_mutation_site(pdb_ids, chain_ids, group_numbers, var_ids, 'STI', 8.0);

In [16]:
spark.stop()